<a href="https://colab.research.google.com/github/lukejoneslj/GeneralConferenceScraper/blob/main/ConferenceScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# You don't need to read all of this if you don't want to. Just "run all" if you want" and wait for it to download.

### Install Required Libraries

This cell installs the necessary Python libraries for web scraping, working with HTML content, and data manipulation.

- `requests`: Used to fetch HTML content from web pages.
- `beautifulsoup4`: Parses and extracts content from the HTML.
- `PyPDF2`: If you need to work with PDF files.


In [ ]:
!pip install requests beautifulsoup4 PyPDF2



### Web Scraping Conference Talks

This block contains helper functions for scraping LDS General Conference talks from the Church's website.

- `get_soup()`: Sends a request and parses the HTML from a given URL.
- `is_decade_page()`: Identifies if the URL is a decade selection page.
- `scrape_conference_pages()`: Fetches URLs for each conference (April/October) from the main page.
- `scrape_talk_urls()`: Retrieves URLs for individual talks from a specific conference.
- `scrape_talk_data()`: Scrapes the detailed data for each talk, such as title, speaker, calling, year, and season.


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import unicodedata
import time

def get_soup(url):
    """Create a tree structure (BeautifulSoup) out of a GET request's HTML."""
    try:
        r = requests.get(url, allow_redirects=True)
        r.raise_for_status()
        print(f"Successfully fetched {r.url}")
        return BeautifulSoup(r.content, "html5lib")
    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

def is_decade_page(url):
    """Check if a page is a decade selection page."""
    return bool(re.search(r"/study/general-conference/\d{4}\d{4}", url))

def scrape_conference_pages(main_page_url):
    """Retrieve a list of URLs for each conference (year/month) from the main page."""
    soup = get_soup(main_page_url)
    if soup is None:
        print(f"Failed to fetch content from {main_page_url}")
        return []

    all_conference_links = []

    # Find all the links to individual conferences or decades
    links = [
        "https://www.churchofjesuschrist.org" + a["href"]
        for a in soup.find_all("a", href=True)
        if re.search(r"/study/general-conference/(\d{4}/(04|10)|\d{4}\d{4})", a["href"])
    ]

    for link in links:
        if is_decade_page(link):
            # Handle decade page
            decade_soup = get_soup(link)
            if decade_soup:
                year_links = [
                    "https://www.churchofjesuschrist.org" + a["href"]
                    for a in decade_soup.find_all("a", href=True)
                    if re.search(r"/study/general-conference/\d{4}/(04|10)", a["href"])
                ]
                all_conference_links.extend(year_links)
        else:
            all_conference_links.append(link)

    print(f"Total conference links found: {len(all_conference_links)}")
    print("Sample conference links:", all_conference_links[:5])
    return all_conference_links

def scrape_talk_urls(conference_url):
    """Retrieve a list of URLs for each talk in a specific conference."""
    soup = get_soup(conference_url)
    if soup is None:
        return []

    talk_links = [
        "https://www.churchofjesuschrist.org" + a["href"]
        for a in soup.find_all("a", href=True)
        if re.search(r"/study/general-conference/\d{4}/(04|10)/.*", a["href"])
    ]

    # Remove duplicate links and session links
    talk_links = list(set(talk_links))
    talk_links = [link for link in talk_links if not link.endswith("session?lang=eng")]

    print(f"Found {len(talk_links)} talk links in {conference_url}")
    if talk_links:
        print("Sample talk links:", talk_links[:3])
    return talk_links

def scrape_talk_data(url):
    """Scrapes a single talk for data such as: title, conference, calling, speaker, content."""
    try:
        soup = get_soup(url)
        if soup is None:
            return {}

        title_tag = soup.find("h1", {"id": "title1"})
        title = title_tag.text.strip() if title_tag else "No Title Found"

        conference_tag = soup.find("p", {"class": "subtitle"})
        conference = conference_tag.text.strip() if conference_tag else "No Conference Found"

        author_tag = soup.find("p", {"class": "author-name"})
        speaker = author_tag.text.strip() if author_tag else "No Speaker Found"

        calling_tag = soup.find("p", {"class": "author-role"})
        calling = calling_tag.text.strip() if calling_tag else "No Calling Found"

        content_array = soup.find("div", {"class": "body-block"})
        content = "\n\n".join(paragraph.text.strip() for paragraph in content_array.find_all("p")) if content_array else "No Content Found"

        footnotes = "\n".join(
            f"{idx}. {note.text.strip()}" for idx, note in enumerate(soup.find_all("li", {"class": "study-note"}), start=1)
        ) if soup.find_all("li", {"class": "study-note"}) else "No Footnotes Found"

        year = re.search(r'/(\d{4})/', url).group(1)
        season = "April" if "/04/" in url else "October"

        return {
            "title": title,
            "speaker": speaker,
            "calling": calling,
            "conference": conference,
            "year": year,
            "season": season,
            "url": url,
            "talk": content,
            "footnotes": footnotes,
        }
    except Exception as e:
        print(f"Failed to scrape {url}: {e}")
        return {}

def main_scrape_process():
    main_url = "https://www.churchofjesuschrist.org/study/general-conference?lang=eng"
    conference_urls = scrape_conference_pages(main_url)

    all_talk_urls = []
    for conference_url in conference_urls:
        all_talk_urls.extend(scrape_talk_urls(conference_url))

    print(f"Total talks found: {len(all_talk_urls)}")

    conference_talks = []
    for i, url in enumerate(all_talk_urls):
        print(f"Scraping talk {i+1}/{len(all_talk_urls)}: {url}")
        talk_data = scrape_talk_data(url)
        if talk_data:
            conference_talks.append(talk_data)

    conference_df = pd.DataFrame(conference_talks)

    for col in conference_df.columns:
        conference_df[col] = conference_df[col].apply(lambda x: unicodedata.normalize("NFD", x) if isinstance(x, str) else x)
        conference_df[col] = conference_df[col].apply(lambda x: x.replace("\t", "") if isinstance(x, str) else x)

    conference_df.to_csv("conference_talks.csv", index=False)
    print("Scraping complete. Data saved to 'conference_talks.csv'.")

    conference_df.to_json("conference_talks.json", orient="records", indent=4)
    print("Data also saved to 'conference_talks.json'.")

start = time.time()
main_scrape_process()
end = time.time()
print(f"Total time taken: {end - start} seconds")

In [4]:
from google.colab import files
files.download('conference_talks.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Clean Conference Talks Data

This code block performs several cleaning operations on the scraped data:
- Removes rows with "Church Auditing Department" in the calling.
- Removes unnecessary columns, like "conference" and "footnotes."
- Standardizes titles for speakers and callings (e.g., "Quorum of the 12" and "Seventy").
- Ensures uniformity across callings and speaker titles.
- Saves the cleaned data to a CSV file for easier analysis and sharing.

You can run this after scraping the data to ensure consistency and uniformity across the dataset.


In [ ]:
import pandas as pd
import re

def clean_conference_data(file_path):
    # Load the CSV file
    df = pd.read_csv(file_path)

    # Step 1: Remove rows with "Church Auditing Department" in the "calling" column
    df = df[~df['calling'].str.contains("Church Auditing Department", na=False)]

    # Step 2: Remove the "conference" column
    df = df.drop(columns=['conference'])

    # Step 3: Modify callings to standardize common names
    # Replace "Quorum of the Twelve Apostles", "twelve", or "12" with "Quorum of the 12"
    df['calling'] = df['calling'].str.replace(r'Q_of_12|twelve|12|Council of the 12', 'Quorum of the 12', regex=True)

    # Replace all variations of "Seventy" (including Quorum, Assistant, Former, Released) with "Seventy"
    df['calling'] = df['calling'].str.replace(r'Q_of_70|70|Assistant to the Q_of_12|First Council of the Seventy|Presidency of the First Q_of_70|Emeritus member of the Seventy|Released Member of the Seventy', 'Seventy', regex=True)

    # Standardize "President of the Church" across all variations
    df['calling'] = df['calling'].str.replace(r'President of The Church of Jesus Christ of Latter-day Saints|President of the Church', 'President of the Church', regex=True)

    # Step 4: Remove rows with "Presented by" in the speaker column
    df = df[~df['speaker'].str.contains("Presented by", na=False)]

    # Step 5: Remove "morning", "afternoon", "evening" session titles (non-talk rows)
    df = df[~df['title'].str.contains(r'morning|afternoon|evening', case=False, na=False)]

    # Step 6: Standardize "Former" or variations in calling titles
    df['calling'] = df['calling'].str.replace(r'Former member of the Seventy', 'Seventy', regex=True)

    # Step 7: Remove "Elder", "President", "Sister", "Brother" from speaker names
    df['speaker'] = df['speaker'].str.replace(r'Elder|President|Sister|Brother', '', regex=True).str.strip()

    # Step 8: Remove the "footnotes" column as it was not needed
    if 'footnotes' in df.columns:
        df = df.drop(columns=['footnotes'])

    # Save the cleaned data to CSV
    df.to_csv("cleaned_conference_talks.csv", index=False)
    print("Data cleaned and saved to 'cleaned_conference_talks.csv'.")

# Example usage
clean_conference_data('conference_talks.csv')
